<a href="https://colab.research.google.com/github/BenjaAR/LAB_14_MD_AQUINO/blob/main/Laboratorio_14_MD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LABORATORIO 14: CONCEPTOS PRELIMINARES MULTICLASIFICADORES
## INTEGRANTES:
- AQUINO REYES BENJY ALVARO
- HUAMAN QUISPE ABRAHAM SEBASTIAN JONATHAN

### 1.- Tomando la información disponible en el repositorio UCI Machine Learning https://archive.ics.uci.edu/ml/datasets/Productivity+Prediction+of+Garment+Employees, pero eliminando previamente a la variable ‘date’ y tomando a la variable ‘actual_productivity’ como variable predictora, haga lo siguiente:

### A.- Realice el preprocesamiento de la información que incluya el análisis de datos faltantes y tratamiento de outliers a nivel univariado y multivariado. Además, convierta las variables categóricas a dummies y aplique un escalamiento a las variables numéricas.

In [ ]:
#pip install ucimlrepo

In [ ]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
productivity_prediction_of_garment_employees = fetch_ucirepo(id=597)

# data (as pandas dataframes)
X = productivity_prediction_of_garment_employees.data.features
y = productivity_prediction_of_garment_employees.data.targets

# metadata
print(productivity_prediction_of_garment_employees.metadata)

# variable information
print(productivity_prediction_of_garment_employees.variables)


{'uci_id': 597, 'name': 'Productivity Prediction of Garment Employees', 'repository_url': 'https://archive.ics.uci.edu/dataset/597/productivity+prediction+of+garment+employees', 'data_url': 'https://archive.ics.uci.edu/static/public/597/data.csv', 'abstract': 'This dataset includes important attributes of the garment manufacturing process and the productivity of the employees which had been collected manually and also been validated by the industry experts.', 'area': 'Business', 'tasks': ['Classification', 'Regression'], 'characteristics': ['Multivariate', 'Time-Series'], 'num_instances': 1197, 'num_features': 14, 'feature_types': ['Integer', 'Real'], 'demographics': [], 'target_col': ['actual_productivity'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 2020, 'last_updated': 'Mon Feb 26 2024', 'dataset_doi': '10.24432/C51S6D', 'creators': [], 'intro_paper': {'ID': 399, 'type': 'NATIVE', 'title': 'Mining the productivity dat

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Cargar datos desde UCI
url = 'https://archive.ics.uci.edu/static/public/597/data.csv'
data = pd.read_csv(url)
data

,date,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,1/1/2015,Quarter1,sweing,Thursday,8,0.80,26.16,1108.0,7080,98,0.0,0,0,59.0,0.940725
1,1/1/2015,Quarter1,finishing,Thursday,1,0.75,3.94,NaN,960,0,0.0,0,0,8.0,0.886500
2,1/1/2015,Quarter1,sweing,Thursday,11,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
3,1/1/2015,Quarter1,sweing,Thursday,12,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
4,1/1/2015,Quarter1,sweing,Thursday,6,0.80,25.90,1170.0,1920,50,0.0,0,0,56.0,0.800382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,3/11/2015,Quarter2,finishing,Wednesday,10,0.75,2.90,NaN,960,0,0.0,0,0,8.0,0.628333
1193,3/11/2015,Quarter2,finishing,Wednesday,8,0.70,3.90,NaN,960,0,0.0,0,0,8.0,0.625625
1194,3/11/2015,Quarter2,finishing,Wednesday,7,0.65,3.90,NaN,960,0,0.0,0,0,8.0,0.625625
1195,3/11/2015,Quarter2,finishing,Wednesday,9,0.75,2.90,NaN,1800,0,0.0,0,0,15.0,0.505889


In [ ]:
# Eliminar variable 'date'
data = data.drop(columns=['date'])

# Ver datos faltantes y outliers
print("Datos faltantes por columna:\n", data.isnull().sum())
print("\nResumen estadístico:\n", data.describe())

# Seleccionar solo columnas numéricas para el tratamiento de outliers
numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns

# Verificar si hay múltiples columnas numéricas
if len(numeric_cols) > 1:
    # Tratamiento de outliers para múltiples columnas
    Q1 = data[numeric_cols].quantile(0.25)
    Q3 = data[numeric_cols].quantile(0.75)
    IQR = Q3 - Q1

    outlier_mask = ((data[numeric_cols] < (Q1 - 1.5 * IQR)) | (data[numeric_cols] > (Q3 + 1.5 * IQR)))
    data = data[~outlier_mask.any(axis=1)]
else:
    # Tratamiento de outliers para una sola columna numérica
    col_name = numeric_cols[0]
    Q1 = data[col_name].quantile(0.25)
    Q3 = data[col_name].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    data = data[(data[col_name] >= lower_bound) & (data[col_name] <= upper_bound)]

Datos faltantes por columna:
 quarter                    0
department                 0
day                        0
team                       0
targeted_productivity      0
smv                        0
wip                      506
over_time                  0
incentive                  0
idle_time                  0
idle_men                   0
no_of_style_change         0
no_of_workers              0
actual_productivity        0
dtype: int64

Resumen estadístico:
               team  targeted_productivity          smv           wip  \
count  1197.000000            1197.000000  1197.000000    691.000000   
mean      6.426901               0.729632    15.062172   1190.465991   
std       3.463963               0.097891    10.943219   1837.455001   
min       1.000000               0.070000     2.900000      7.000000   
25%       3.000000               0.700000     3.940000    774.500000   
50%       6.000000               0.750000    15.260000   1039.000000   
75%       9.000000      

In [ ]:
# Convertir variables categóricas a dummies
data = pd.get_dummies(data, columns=['department', 'day'])

# Separar variable predictora ('actual_productivity')
X = data.drop(columns=['actual_productivity'])
y = data['actual_productivity']

# Escalar variables numéricas
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.select_dtypes(include=['float64', 'int64']))
X[X.select_dtypes(include=['float64', 'int64']).columns] = X_scaled

### B.- Separe los datos en entrenamiento (80%) y prueba (20%) y, realice un ensamblaje utilizando las técnicas de voting, bagging, boosting y stacking para los modelos k-NN, SVM, regresión lineal, árbol de clasificación y Random Forest definiendo distintos hiperparámetros para cada modelo.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, VotingRegressor, BaggingRegressor, AdaBoostRegressor, StackingRegressor

# Modelos base
models = [
    ('k-NN', KNeighborsRegressor(n_neighbors=5)),
    ('SVM', SVR(C=1.0, gamma='scale')),
    ('RL', LinearRegression()),
    ('DTR', DecisionTreeRegressor(max_depth=5)),
    ('RFR', RandomForestRegressor(n_estimators=100, random_state=42))
]

# Ensamblajes
voting = VotingRegressor(estimators=models[:3])  # Usar primeros 3 modelos para voting
bagging = BaggingRegressor(estimator=DecisionTreeRegressor(), n_estimators=50)  # Cambiado a 'estimator'
boosting = AdaBoostRegressor(estimator=RandomForestRegressor(), n_estimators=50)  # También cambiado aquí
stacking = StackingRegressor(estimators=models, final_estimator=LinearRegression())

ensembles = [
    ('Voting', voting),
    ('Bagging', bagging),
    ('Boosting', boosting),
    ('Stacking', stacking)
]

### C.- Utilice las métricas error cuadrático medio, raíz del error cuadrático medio, error absoluto medio y coeficiente de determinación para elegir el mejor modelo que prediga a la variable ‘actual_productivity’, justificando su respuesta.


In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_validate
import numpy as np

# 1. Preprocesamiento robusto
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# 2. Configuración de modelos más robusta
models = [
    ('k-NN', KNeighborsRegressor(n_neighbors=5)),
    ('SVM', SVR(C=1.0, gamma='scale', kernel='rbf')),
    ('RL', LinearRegression()),
    ('DTR', DecisionTreeRegressor(max_depth=5, min_samples_split=5)),
    ('RFR', RandomForestRegressor(n_estimators=100, min_samples_split=5, random_state=42))
]

# 3. Crear pipelines completos
model_pipelines = []
for name, model in models:
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    model_pipelines.append((name, pipeline))

# 4. Evaluación con manejo de errores
results = {}
for name, pipeline in model_pipelines:
    try:
        cv_results = cross_validate(
            pipeline,
            X_train,
            y_train,
            cv=5,
            scoring=scoring,
            error_score='raise'  # Para ver errores detallados
        )

        mse = -cv_results['test_mse'].mean()
        rmse = -cv_results['test_rmse'].mean()
        mae = -cv_results['test_mae'].mean()
        r2 = cv_results['test_r2'].mean()

        results[name] = {'MSE': mse, 'RMSE': rmse, 'MAE': mae, 'R2': r2}

    except Exception as e:
        print(f"Error con {name}: {str(e)}")
        results[name] = {'MSE': np.nan, 'RMSE': np.nan, 'MAE': np.nan, 'R2': np.nan}

# Mostrar resultados
for model, metrics in results.items():
    print(f"\n{model}:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.4f}")


k-NN:
  MSE: 0.0145
  RMSE: 0.1200
  MAE: 0.0791
  R2: 0.2957

SVM:
  MSE: 0.0146
  RMSE: 0.1204
  MAE: 0.0879
  R2: 0.2913

RL:
  MSE: 0.0147
  RMSE: 0.1209
  MAE: 0.0868
  R2: 0.2861

DTR:
  MSE: 0.0130
  RMSE: 0.1136
  MAE: 0.0763
  R2: 0.3698

RFR:
  MSE: 0.0106
  RMSE: 0.1030
  MAE: 0.0634
  R2: 0.4824


In [ ]:
best_model = min(results.items(), key=lambda x: x[1]['RMSE'])
print(f"\nMejor modelo: {best_model[0]} con RMSE = {best_model[1]['RMSE']:.4f}")


Mejor modelo: RFR con RMSE = 0.1030
